In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df=pd.read_csv('data/nsl-kdd.csv')
df

In [ ]:
df.columns

In [ ]:
df.rename(columns = {'[duration':'duration',' label':'label',' protocol_type':'protocol_type',' service':'service',' flag':'flag'}, inplace = True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['protocol_type'] = le.fit_transform(df['protocol_type'])
df['service'] = le.fit_transform(df['service'])
df['flag'] = le.fit_transform(df['flag'])
df['label'] = le.fit_transform(df['label'])



df.drop(" difficulty_level]", axis=1, inplace=True)

df.head(10)

In [ ]:
x_columns = df.columns.drop('label')
x = df[x_columns].values
dummies = pd.get_dummies(df['label']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

In [ ]:
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

n_samples_max = 200
x_train = x_train[0:n_samples_max]
y_train = y_train[0:n_samples_max]
x_test = x_test[0:n_samples_max]
y_test = y_test[0:n_samples_max]


In [ ]:
# pip install adversarial-robustness-toolbox

In [ ]:
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import SklearnClassifier
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
                           class_weight='balanced', random_state=None, solver='lbfgs', max_iter=100, 
                           multi_class='ovr', verbose=0, warm_start=False, n_jobs=None)

In [ ]:
model.fit(x_train,y_train)
accuracy_before_attack =  model.score(x_test, y_test)
print(accuracy_before_attack)

In [ ]:
art_classifier = SklearnClassifier(model)

In [ ]:
pgd = ProjectedGradientDescent(estimator=art_classifier, norm=np.inf, eps=.3, eps_step=0.1, max_iter=20, 
                               targeted=False, num_random_init=0, batch_size=100)

In [ ]:
x_train_adv = pgd.generate(x_train)
x_test_adv = pgd.generate(x_test)

In [ ]:
accuracy_after_attack = model.score(x_test_adv,y_test)
print(accuracy_after_attack )

In [ ]:
print("Accuracy before attack:", accuracy_before_attack)
print("Accuracy after attack:", accuracy_after_attack)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print("Accuracy before attack:", accuracy_before_attack)
print("Accuracy after attack:", accuracy_after_attack)

accuracy_data = [accuracy_before_attack, accuracy_after_attack]
labels = ['Before Attack', 'After Attack']
x_pos = np.arange(len(labels))

plt.bar(x_pos, accuracy_data, align='center', alpha=0.5)
plt.xticks(x_pos, labels)
plt.ylabel('Accuracy')
plt.title('Accuracy Before and After Attack')
plt.savefig("figure.png")
plt.show()


In [ ]:
test_prediction1 = model.predict(x_test)
print(test_prediction1)

In [ ]:
test_prediction2 = model.predict(x_test_adv)
print(test_prediction2)